# Trabajo Practico 2

In [5]:
import numpy as np

## Ejercicio 1

Simule el siguiente problema.
Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
El tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media 𝜇
Considerar 𝜇 = 1, 2 𝑦 4 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠

Realizar 100 simulaciones de cada modelo, con 100000 solicitudes procesadas, y determinar:
- El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión se
cae por timeout).
- La fracción de las solicitudes que no esperaron para ser procesadas.
- La tasa de finalización de consultas (consultas finalizadas por segundo)
- ¿Qué solución le recomienda? Justifique

In [63]:
def tiempos_de_arribo(media, cantidad_ensayos):
    z = np.random.exponential(media, cantidad_ensayos)
    return np.concatenate(([0], np.cumsum(z)), axis=None)

#### Alternativa 1
Se utilizan 2 bases de datos distribuidas.
Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base A y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la
base de datos B.
El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias,
𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 0,95 𝑠𝑒𝑔 respectivamente.

In [64]:
def tiempos_en_base(arribos, media):
    t_actual = 0
    tiempos_espera = []
    tiempos_procesado = []
    for t_arribo in arribos:
        t_base = np.random.exponential(media)
        if t_arribo < t_actual:
            t_espera = t_actual - t_arribo
            tiempos_espera.append(t_espera)
            t_procesado = t_arribo + t_espera + t_base
            tiempos_procesado.append(t_procesado)
            t_actual += t_base
        else:
            t_espera = 0
            tiempos_espera.append(t_espera)
            t_procesado = t_espera + t_base
            tiempos_procesado.append(t_procesado)
            t_actual = t_arribo + t_base
    return tiempos_espera, tiempos_procesado

In [65]:
def alternativa_1(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    proba_a = 0.6
    tiempos_a = []
    tiempos_b = []
    for t in tiempos_arribo:
        bdd = np.random.uniform()
        t_p = t
        t_e = 0
        if bdd < proba_a:
            tiempos_a.append(t)
        else:
            tiempos_b.append(t)
    tiempos_espera_a,tiempos_procesado_a = tiempos_en_base(tiempos_a, 0.7)
    tiempos_espera_b,tiempos_procesado_b = tiempos_en_base(tiempos_b, 0.95)
    return tiempos_a + tiempos_b, tiempos_procesado_a + tiempos_procesado_b, tiempos_espera_a + tiempos_espera_b

In [66]:
tiempos_arribo, tiempos_procesado, tiempos_espera = alternativa_1(4, 100000)

In [26]:
tiempo_medio = sum(tiempos_espera)/len(tiempos_espera)

In [27]:
tiempo_medio

0.08756890505565972

In [82]:
def simular(media_arribos, cantidad_simulaciones, muestra, alternativa):
    tiempos_medios_espera_resultado = []
    fraccion_sin_espera_resultado = []
    tasa_finalizacion_resultado = []
    for i in range(cantidad_simulaciones):
        tiempos_arribo, tiempos_procesado, tiempos_espera = alternativa(media_arribos, muestra)
        tiempos_medios_espera_resultado.append(sum(tiempos_espera)/muestra)
        sin_espera = list(filter(lambda x: x==0, tiempos_espera))
        fraccion_sin_espera = len(sin_espera) / muestra
        fraccion_sin_espera_resultado.append(fraccion_sin_espera)
        tasa_finalizacion = muestra / max(tiempos_procesado)
        tasa_finalizacion_resultado.append(tasa_finalizacion)
    tiempo_medio_total = sum(tiempos_medios_espera_resultado)/len(tiempos_medios_espera_resultado)
    fraccion_sin_espera_total = sum(fraccion_sin_espera_resultado)/len(fraccion_sin_espera_resultado)
    tasa_finalizacion_total = sum(tasa_finalizacion_resultado)/len(tasa_finalizacion_resultado)
    return tiempo_medio_total, fraccion_sin_espera_total, tasa_finalizacion_total

In [83]:
def mostrar_resultado(resultado):
    tiempo_medio_espera, fraccion_sin_espera, tasa_finalizacion = resultado
    print("Tiempo medio de espera: {} segundos".format(tiempo_medio_espera))
    print("Fraccion de solicitudes sin esperar: {}".format(fraccion_sin_espera))
    print("Tasa de finalizacion: {} solicitudes finalizadas por segundo".format(tasa_finalizacion))

In [84]:
mostrar_resultado(simular(4, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.0892841631057477 segundos
Fraccion de solicitudes sin esperar: 0.8988531000000002
Tasa de finalizacion: 0.24990174994915684 solicitudes finalizadas por segundo


In [85]:
mostrar_resultado(simular(2, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.20023391656873377 segundos
Fraccion de solicitudes sin esperar: 0.7982427000000001
Tasa de finalizacion: 0.49975768117888014 solicitudes finalizadas por segundo


In [86]:
mostrar_resultado(simular(1, 100, 100000, alternativa_1))

Tiempo medio de espera: 0.536807224366776 segundos
Fraccion de solicitudes sin esperar: 0.5960978
Tasa de finalizacion: 0.9999837397968336 solicitudes finalizadas por segundo


#### Alternativa 2
Utilizar 1 base de datos central.
En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜

In [88]:
def alternativa_2(media_arribos, solicitudes_procesadas):
    tiempos_arribo = tiempos_de_arribo(media_arribos, solicitudes_procesadas)
    tiempos_espera,tiempos_procesado = tiempos_en_base(tiempos_arribo, 0.8)
    return tiempos_arribo, tiempos_procesado, tiempos_espera

In [89]:
mostrar_resultado(simular(4, 100, 100000, alternativa_2))

Tiempo medio de espera: 0.20005883338272462 segundos
Fraccion de solicitudes sin esperar: 0.8001481999999999
Tasa de finalizacion: 0.250094255014842 solicitudes finalizadas por segundo


In [90]:
mostrar_resultado(simular(2, 100, 100000, alternativa_2))

Tiempo medio de espera: 0.5335923928684551 segundos
Fraccion de solicitudes sin esperar: 0.5997661999999998
Tasa de finalizacion: 0.5000102721622344 solicitudes finalizadas por segundo


In [91]:
mostrar_resultado(simular(1, 100, 100000, alternativa_2))

Tiempo medio de espera: 3.212825179536111 segundos
Fraccion de solicitudes sin esperar: 0.19969410000000007
Tasa de finalizacion: 1.0000964828727401 solicitudes finalizadas por segundo
